In [2]:
import math
import sys
import numpy as np
from scipy.special import logsumexp

def forward_log(observs, states, sp, tp, ep):
    # 状態 FとLが格納される
    T = {}
    Ts = []
    
    # 初期化
    T["F"] = 0
    T["L"] = -float("inf")
        
    # 再帰
    for ob in observed:
        T = next_log(ob, states, T, tp, ep)
        Ts.append(T)
        
    # 終了処理
    U = {}
    l = [T["F"]+tp["F"]["F"],T["L"]+tp["L"]["F"]]
    U["F"] = logsumexp(l)

    l = [T["F"]+tp["F"]["L"], T["L"]+tp["L"]["L"]]
    U["L"] = logsumexp(l)

    return Ts,U


def next_log(ob, states, T, tp, ep):
    # 状態 FとLが格納される
    U = {}
    
    l = [T["F"]+tp["F"]["F"],T["L"]+tp["L"]["F"]]
    U["F"] = ep["F"][ob]+logsumexp(l)
    
    l = [T["F"]+tp["F"]["L"], T["L"]+tp["L"]["L"]]
    U["L"] = ep["L"][ob]+logsumexp(l)

    return U


def forward_scaling(observed, states, sp, tp, ep):
    T = {}
    Ts = []
    P = 1
    
    # 初期化
    T["F"] = 1
    T["L"] = 0
        
    # 再帰
    for ob in observed:
        T,si = next_scale(ob, states, T, tp, ep)
        P *= si
        Ts.append(T)
        
    # 終了処理
    # これはP

    return Ts,P


def next_scale(ob, states, T, tp, ep):
    # 状態 FとLが格納される
    U = {}
    si = 0
    S = {}
    
    S["F"] = ep["F"][ob]*(T["F"]*tp["F"]["F"]+T["L"]*tp["L"]["F"])
    
    S["L"] = ep["L"][ob]*(T["F"]*tp["F"]["L"]+T["L"]*tp["L"]["L"])
    
    si = S["F"]+S["L"]
    
    U["F"] = (1/si)*S["F"]
    U["L"] = (1/si)*S["L"]

    return U,si

In [3]:
f = open('observed.txt')
observed = f.read()
f.close()

########################## log
states = ("F", "L")
start_prob = {"F": math.log(0.5), "L": math.log(0.5)}
transit_prob = {"F": {"F": math.log(0.95), "L": math.log(0.05)},
                "L": {"F": math.log(0.1), "L": math.log(0.9)}}
emission_prob = {'F': {'1': math.log(1/6), '2': math.log(1/6), '3': math.log(1/6), '4': math.log(1/6), '5': math.log(1/6), '6': math.log(1/6)},
                 'L': {'1': math.log(1/10), '2': math.log(1/10), '3': math.log(1/10), '4': math.log(1/10), '5': math.log(1/10), '6': math.log(1/2)}}

_,logP = forward_log(observed, states,
                      start_prob, transit_prob, emission_prob)


print("周辺化確率:{}".format(logP))



########################## scale
states = ("F", "L")
start_prob = {"F": 0.5, "L": 0.5}
transit_prob = {"F": {"F": 0.95, "L": 0.05},
                "L": {"F": 0.1, "L": 0.9}}
emission_prob = {'F': {'1': 1/6, '2': 1/6, '3': 1/6, '4': 1/6, '5': 1/6, '6': 1/6},
                 'L': {'1': 1/10, '2': 1/10, '3': 1/10, '4': 1/10, '5': 1/10, '6': 1/2}}

_,P = forward_scaling(observed, states,
                          start_prob, transit_prob, emission_prob)

print("周辺化確率:{}".format(math.log(P)))

FileNotFoundError: [Errno 2] No such file or directory: 'observed.txt'

- 結果

- log
    -516.6013987084424
    
- scale
    -516.4838926559529